In [92]:
from music21 import converter, note, chord, key, meter, instrument
import numpy as np
import os
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [106]:
def get_midi_features(midi_file, instrument_family_map):
    try:
        # Parse the MIDI file into a music21 stream object
        score = converter.parse(midi_file)

        # Flatten the stream to simplify traversal (all elements at the same level)
        flat = score.flatten()

        # Analyze the global key signature of the piece
        key_signature = flat.analyze('key')
        # Encode key as integer 0–11 for tonic pitch class + 12 if minor mode to distinguish modality
        key_index = key_signature.tonic.pitchClass + (12 if key_signature.mode == 'minor' else 0)

        # Get the first time signature encountered in the score, default to 4/4 if missing
        time_signature = flat.recurse().getElementsByClass(meter.TimeSignature).first()
        time_sig = str(time_signature.ratioString) if time_signature else '4/4'

        features = []  # List to hold feature dictionaries for each musical event
        previous_pitch = None  # To compute interval to previous note

        # Group notes and chords by their onset time to detect polyphony (simultaneous notes)
        offset_dict = {}
        for element in flat.notes:
            offset_dict.setdefault(element.offset, []).append(element)

        # Define helper function to extract instrument name and family from context
        def extract_instrument(element):
            instr_name = "Unknown"
            instr_family = "Other"
            instr = element.getContextByClass(instrument.Instrument)
            if instr and hasattr(instr, 'midiProgram'):
                try:
                    # Attempt to recover instrument name from MIDI program
                    instr_obj = instrument.instrumentFromMidiProgram(instr.midiProgram)
                    instr_name = instr_obj.instrumentName
                    # Use family mapping if provided
                    instr_family = instrument_family_map.get(instr_name, "Other") if instrument_family_map else "Other"
                except Exception:
                    pass  # Fallback to defaults if resolution fails
            return instr_name, instr_family

        # Iterate over each onset time and the notes/chords sounding at that time
        for offset, simultaneous_notes in offset_dict.items():
            polyphony = len(simultaneous_notes)  # Number of overlapping events at this onset

            for element in simultaneous_notes:
                # Extract instrument information from the context of the element
                instr_name, instr_family = extract_instrument(element)

                if isinstance(element, note.Note):
                    # Extract basic pitch and timing features
                    pitch = element.pitch.midi  # MIDI pitch number (0–127)
                    pitch_class = element.pitch.pitchClass  # Pitch class modulo 12
                    duration = element.quarterLength  # Note duration in quarter lengths

                    # Calculate melodic interval to previous note; 0 if this is the first note
                    interval_to_prev = abs(pitch - previous_pitch) if previous_pitch is not None else 0
                    previous_pitch = pitch  # Update for next comparison

                    # Structural location of the event in the score
                    measure_number = element.measureNumber if element.measureNumber is not None else 0

                    # Whether the note is part of a chord (set to 0 here; see chord block for 1)
                    is_chord_tone = 0

                    # Append the extracted features for this note event
                    features.append({
                        'pitch': pitch,
                        'pitch_class': pitch_class,
                        'onset': offset,
                        'duration': duration,
                        'interval_to_prev': interval_to_prev,
                        'polyphony': polyphony,
                        'is_chord_tone': is_chord_tone,
                        'key': key_index,
                        'time_signature': time_sig,
                        'measure': measure_number,
                        'instrument': instr_name,
                        'instrument_family': instr_family
                    })

                elif isinstance(element, chord.Chord):
                    # Extract pitch and duration for each note in the chord
                    duration = element.quarterLength
                    measure_number = element.measureNumber if element.measureNumber is not None else 0
                    is_chord_tone = 1  # Chord members are marked as chord tones

                    for pitch in [p.midi for p in element.pitches]:
                        pitch_class = pitch % 12
                        interval_to_prev = abs(pitch - previous_pitch) if previous_pitch is not None else 0
                        previous_pitch = pitch

                        # Append feature dict for each note in the chord separately
                        features.append({
                            'pitch': pitch,
                            'pitch_class': pitch_class,
                            'onset': offset,
                            'duration': duration,
                            'interval_to_prev': interval_to_prev,
                            'polyphony': polyphony,
                            'is_chord_tone': is_chord_tone,
                            'key': key_index,
                            'time_signature': time_sig,
                            'measure': measure_number,
                            'instrument': instr_name,
                            'instrument_family': instr_family
                        })

        return features

    except Exception as e:
        # Catch and report any parsing or extraction errors gracefully
        print(f"[ERROR] Could not parse {midi_file}: {e}")
        return []

In [110]:
# Add instrument family to each event for higher-level grouping (e.g., strings, winds, percussion)
# This enables downstream analysis of timbral or orchestration patterns across genres or pieces

instrument_family_map = {
    # Keyboard
    'Piano': 'Keyboard',
    'Celesta': 'Keyboard',
    'Electric Organ': 'Keyboard',
    'Organ': 'Keyboard',
    'Harpsichord': 'Keyboard',

    # Guitar
    'Acoustic Guitar': 'Guitar',
    'Electric Guitar': 'Guitar',

    # Bass
    'Acoustic Bass': 'Bass',
    'Electric Bass': 'Bass',
    'Fretless Bass': 'Bass',
    'Contrabass': 'Bass',

    # Strings
    'Violoncello': 'Strings',
    'Violin': 'Strings',
    'Viola': 'Strings',
    'Double Bass': 'Strings',
    'StringInstrument': 'Strings',

    # Brass
    'Trumpet': 'Brass',
    'Trombone': 'Brass',
    'French Horn': 'Brass',
    'Tuba': 'Brass',

    # Woodwind
    'Clarinet': 'Woodwind',
    'Bassoon': 'Woodwind',
    'Recorder': 'Woodwind',
    'Piccolo': 'Woodwind',
    'Flute': 'Woodwind',
    'Whistle': 'Woodwind',

    # Percussion
    'Timpani': 'Percussion',
    'Taiko': 'Percussion',
    'Marimba': 'Percussion',
    'Glockenspiel': 'Percussion',
    'Drums': 'Percussion',

    # Voice
    'Voice': 'Voice',
    'Choir': 'Voice',
    'Vocals': 'Voice',
    'Background Vocals': 'Voice',

    # Electronic
    'Sampler': 'Electronic',
    'Synthesizer': 'Electronic',
    'Electric Piano': 'Electronic',
    'Electric Organ': 'Electronic',

    # Other / Catch-all
    'Bagpipes': 'Other',
    'Ocarina': 'Other',
    'Unknown': 'Other',
}

In [108]:
# Define the path to the root folder containing genre-labeled subfolders of MIDI files
root_path = 'data'

# Initialize an empty list to collect feature dictionaries across all files and genres
all_features = []

# Traverse each genre subfolder in the dataset
for genre in os.listdir(root_path):
    genre_path = os.path.join(root_path, genre)
    
    # Skip non-directory files (e.g., stray files in the root directory)
    if not os.path.isdir(genre_path):
        continue

    # Process each MIDI file within the current genre folder
    for file in os.listdir(genre_path):
        if not file.lower().endswith('.mid'):
            continue  # Ignore non-MIDI files

        file_path = os.path.join(genre_path, file)

        # Extract symbolic musical features from the current MIDI file
        features = get_midi_features(file_path, instrument_family_map)

        # Append genre label and filename metadata to each feature row
        for feature in features:
            feature['genre'] = genre
            feature['filename'] = file
            all_features.append(feature)

# Aggregate all extracted features into a structured pandas DataFrame
df = pd.DataFrame(all_features)

# Display the first few rows for inspection
df.head()

,pitch,pitch_class,onset,duration,interval_to_prev,polyphony,is_chord_tone,key,time_signature,measure,instrument,instrument_family,genre,filename
0,27,3,4.0,0.75,0,4,0,3,4/4,2,Sampler,Electronic,pop,Come On Over (All I Want Is You).mid
1,39,3,4.0,0.75,12,4,0,3,4/4,2,Sampler,Electronic,pop,Come On Over (All I Want Is You).mid
2,27,3,4.0,0.75,12,4,0,3,4/4,2,Sampler,Electronic,pop,Come On Over (All I Want Is You).mid
3,39,3,4.0,0.75,12,4,0,3,4/4,2,Sampler,Electronic,pop,Come On Over (All I Want Is You).mid
4,79,7,5.0,0.5,40,2,0,3,4/4,2,Sampler,Electronic,pop,Come On Over (All I Want Is You).mid


In [109]:
df.instrument_family.unique()

array(['Electronic', 'Keyboard', 'Guitar', 'Bass', 'Voice', 'Woodwind',
       'Strings', 'Other', 'Percussion', 'Brass'], dtype=object)

In [96]:
## NEXT STEP: CHECK EVERY OTHER CATEGORY

AttributeError: 'str' object has no attribute 'mode'